In [1]:
import asyncio
from scipy import optimize
from tq42.organization import list_all as list_all_organizations
from tq42.project import list_all as list_all_projects
from tq42.experiment import list_all as list_all_experiments
from tq42.channel import Channel, Ask, Tell
from tq42.client import TQ42Client
from tq42.algorithm import AlgorithmProto
from tq42.compute import HardwareProto
import numpy as np
import OptimizationTestFunctions as otf

from tq42.experiment_run import ExperimentRun
from tq42.algorithm import CvaOptParametersProto

In [2]:
#the main function is declared as async because it contains several await expressions that require it to be asynchronous.
async def main():
    with TQ42Client() as client:
        client.login()
        org_list = list_all_organizations(client=client)
        org_sample = org_list[0]

        proj_list = list_all_projects(client=client, organization_id=org_sample.id)
        proj_sample = proj_list[0]
        
        exp_list = list_all_experiments(client=client, project_id=proj_sample.id)
        exp_sample = exp_list[0]

        print(f"Running experiment within: \n Org: {org_sample.id} \n Proj: {proj_sample.id} \n Exp: {exp_sample.id}")

        #create the communication channels which will receive/send the inputs and outputs from/to tetraopt
        #tetraopt to your objective and local objective functions
        objective_func_channel = await Channel.create(client=client)
        local_opt_channel = await Channel.create(client=client)
        print("objective_func_channel type: ", objective_func_channel)
        print("local_opt_channel type: ", local_opt_channel)

        tetra_opt_params = {
            "dimensionality": 2,
            "iteration_number": 2,
            "maximal_rank": 4,
            "points_number": 1,
            "quantization": False,
            "tolerance": 0.0010000000474974513,
            "lower_limits": [0, 0],
            "upper_limits": [9, 9],
            "grid": [10, 10],
             "objective_function_channel_id": objective_func_channel.id,
            "local_optimizer_channel_id": local_opt_channel.id,
            "polling": {"initial_delay": 1.0, "retries": 100, "delay": 1.0, "backoff_factor": 1.1}
        }

        run = ExperimentRun.create(
            client=client,
            algorithm=AlgorithmProto.TETRA_OPT,
            experiment_id=exp_sample.id,
            compute=HardwareProto.SMALL,
            parameters={'parameters': tetra_opt_params, 'inputs': {}}
        )

        print(f"Experiment run id: {run.id}")



In [ ]:
        #The "ask" argument is provided by tetraopt, it is an array with a dimension of n(ask.headers)
        async def objective_function_callback(ask: Ask) -> Tell:    
            dim = len(ask.headers)
            
            #initialize Ackley function to receive n dimension arrays
            func = otf.Ackley(dim)
            y = []

            
            #append the results of the Ackley function to y for all given n dimension parameters
            for parameter in ask.parameters:
                y.append(float(func(np.array(parameter.values))))

            #create and return a Tell object which has the parameters and the results mapped together
            tell = Tell(
                parameters=ask.parameters,
                headers=ask.headers,
                results=y
            )
            return tell

        
        async def local_optimization_function_callback(ask: Ask) -> Tell:
            dim = len(ask.headers)
            func = otf.Ackley(dim)
            y = []
            new_x = []
            for parameter in ask.parameters:
                res = optimize.minimize(func, np.array(parameter.values))
                new_x.append({"values": res.x})
                y.append(float(res.fun))
           
            tell = Tell(
                parameters=ask.parameters,
                headers=ask.headers,
                results=y,
                candidates=new_x
            )

            return tell

In [ ]:
        def success():
            print("One async function done!")

        await asyncio.gather(
            objective_func_channel.connect(
                 callback=objective_function_callback, finish_callback=success, max_duration_in_sec=None, message_timeout_in_sec=500
            ),
            local_opt_channel.connect(
                callback=local_optimization_function_callback, finish_callback=success, max_duration_in_sec=None, message_timeout_in_sec=500
            )
        )

        print(f"Experiment run result: {run.check()}")
        return run

run = await main()
print(run.poll())